In [20]:
#Dance of the Planets

import numpy as np

G = 6.67430e-11 #Gravitational constant(m^3 kg^-1 s^-2)

#things the user will provide

m_1 = float(input("Enter mass of body 1 (in kg): "))
m_2 = float(input("Enter mass of body 2 (in kg): "))
x_1_0 = np.array([float(input("Enter the initial x-coordinate of the body 1(in m): ")), 
               float(input("Enter the initial y-coordinate of the body 1(in m): "))])

x_2_0 = np.array([float(input("Enter the initial x-coordinate of the body 2(in m): ")), 
               float(input("Enter the initial y-coordinate of the body 2(in m): "))])
v_1_0 = np.array([float(input("Enter the initial x-velocity of the body 1 (in m/s): ")), 
               float(input("Enter the initial y-velocity of the body 1(in m/s): "))])

v_2_0 = np.array([float(input("Enter the initial x-velocity of the body 2(in m/s): ")), 
               float(input("Enter the initial y-velocity of the body 2(in m/s): "))])
t = float(input("Enter the time for prediction (seconds): "))

Enter mass of body 1 (in kg):  3e24
Enter mass of body 2 (in kg):  8e23
Enter the initial x-coordinate of the body 1(in m):  0
Enter the initial y-coordinate of the body 1(in m):  0
Enter the initial x-coordinate of the body 2(in m):  1.5e8
Enter the initial y-coordinate of the body 2(in m):  1.5e8
Enter the initial x-velocity of the body 1 (in m/s):  800
Enter the initial y-velocity of the body 1(in m/s):  1500
Enter the initial x-velocity of the body 2(in m/s):  -1200
Enter the initial y-velocity of the body 2(in m/s):  1000
Enter the time for prediction (seconds):  1e6


In [21]:
#calculating coordinates at T using ODE solvers
#I will use odeint for solving this
from scipy.integrate import odeint

#we need to define 2 functions for it

def two_body_ode(y, t, G, m1, m2): #defines the system of ODE which odeint will solve
    # y contains: x1_x, x1_y, v1_x, v1_y, x2_x, x2_y, v2_x, v2_y
    x1 = y[0:2]
    v1 = y[2:4]
    x2 = y[4:6]
    v2 = y[6:8]

    r = x2 - x1            # vector from body 1 to body 2
    dist_cubed = np.linalg.norm(r)**3

    dx1_dt = v1
    dv1_dt = G * m2 * r / dist_cubed
    dx2_dt = v2
    dv2_dt = G * m1 * (-r) / dist_cubed

    return np.concatenate([dx1_dt, dv1_dt, dx2_dt, dv2_dt])

def get_coordinates_at_t(t, y0, G, m1, m2): #uses odeint to integrate the ode over the specified time t
    t_interval = [0, t]
    sol = odeint(two_body_ode, y0, t_interval, args=(G, m1, m2))
    return sol[-1]  # returns an array of length 8

y0 = np.concatenate([x_1_0, v_1_0, x_2_0, v_2_0])

In [32]:
#Dataset generation
import pandas as pd
#for generating dataset I need to find the values at different times.
#for solving ODE I have used odeint.
no_points = 10000 # let dataset have 10000 points
dt = t/no_points # formula for finding dt

#lists for storing data for DataFrame
time_T = []
x_1 = []
y_1 = []
x_2 = []
y_2 = []
v_1_x = []
v_1_y = []
v_2_x = []
v_2_y = []

#making variable for loop
time_current = 0

while time_current < t :
    result = get_coordinates_at_t(time_current, y0, G, m_1, m_2)
    x_1_current, v_1_current, x_2_current, v_2_current = result[0:2], result[2:4], result[4:6], result[6:8]
    

    #store the data using .append()
    time_T.append(time_current)
    x_1.append(x_1_current[0])
    y_1.append(x_1_current[1])
    x_2.append(x_2_current[0])
    y_2.append(x_2_current[1])
    v_1_x.append(v_1_current[0])
    v_1_y.append(v_1_current[1])
    v_2_x.append(v_2_current[0])
    v_2_y.append(v_2_current[1])

    time_current += dt #update the time to be not stuck in infinite loop

#create dataframe
data = pd.DataFrame({
    'T': time_T,
    'x_1': x_1,
    'y_1': y_1,
    'x_2': x_2,
    'y_2': y_2,
    'v_1_x': v_1_x,
    'v_1_y': v_1_y,
    'v_2_x': v_2_x,
    'v_2_y': v_2_y
})

data

,T,x_1,y_1,x_2,y_2,v_1_x,v_1_y,v_2_x,v_2_y
0,0.0,0.000000e+00,0.000000e+00,1.500000e+08,1.500000e+08,800.000000,1500.000000,-1200.000000,1000.000000
1,100.0,8.000420e+04,1.500042e+05,1.498800e+08,1.501000e+08,800.083950,1500.083992,-1200.314813,999.685029
2,200.0,1.600168e+05,3.000168e+05,1.497599e+08,1.501999e+08,800.167998,1500.168167,-1200.629994,999.369375
3,300.0,2.400378e+05,4.500378e+05,1.496399e+08,1.502999e+08,800.252145,1500.252524,-1200.945542,999.053035
4,400.0,3.200672e+05,6.000673e+05,1.495197e+08,1.503997e+08,800.336389,1500.337065,-1201.261459,998.736007
...,...,...,...,...,...,...,...,...,...
9995,999500.0,4.611896e+08,1.733270e+09,2.196389e+08,2.719234e+08,398.065935,1706.502776,307.252745,225.614588
9996,999600.0,4.612294e+08,1.733441e+09,2.196697e+08,2.719460e+08,398.065538,1706.500376,307.254234,225.623592
9997,999700.0,4.612692e+08,1.733612e+09,2.197004e+08,2.719685e+08,398.065141,1706.497975,307.255721,225.632594
9998,999800.0,4.613090e+08,1.733782e+09,2.197311e+08,2.719911e+08,398.064744,1706.495575,307.257209,225.641594


In [33]:
#as we only need to predict x_1, y_1, x_2 and y_2 we can drop v_1_x, v_1_y, v_2_x and v_2_y

data = data.drop( ["v_1_x", "v_1_y", "v_2_x", "v_2_y"], axis = 1) #code to drop columns 

# Save the DataFrame to a CSV file
data.to_csv('data.csv', index=False)

data

,T,x_1,y_1,x_2,y_2
0,0.0,0.000000e+00,0.000000e+00,1.500000e+08,1.500000e+08
1,100.0,8.000420e+04,1.500042e+05,1.498800e+08,1.501000e+08
2,200.0,1.600168e+05,3.000168e+05,1.497599e+08,1.501999e+08
3,300.0,2.400378e+05,4.500378e+05,1.496399e+08,1.502999e+08
4,400.0,3.200672e+05,6.000673e+05,1.495197e+08,1.503997e+08
...,...,...,...,...,...
9995,999500.0,4.611896e+08,1.733270e+09,2.196389e+08,2.719234e+08
9996,999600.0,4.612294e+08,1.733441e+09,2.196697e+08,2.719460e+08
9997,999700.0,4.612692e+08,1.733612e+09,2.197004e+08,2.719685e+08
9998,999800.0,4.613090e+08,1.733782e+09,2.197311e+08,2.719911e+08


In [34]:
#Splitting the data

#before that I'll find max values of each clumn which will be used in normalization of the data for more stable training
T_max = data["T"].max()
x_1_max = data["x_1"].max()
y_1_max = data["y_1"].max()
x_2_max = data["x_2"].max()
y_2_max = data["y_2"].max()

#as this has input only as time I'm not using train_test_split as it might lead to data leakage, instead I'm manually splitting the data
#could use TimeSeriesSplit as well
#refer source - https://tomerkatzav.medium.com/split-time-series-dataset-826b7dc39cd9
train_data_size = int(len(data)*0.7) #spliiting such that 70% is train data, 15% is validation data and rest is test data
val_data_size = int(len(data)*0.15)

train_data = data.iloc[ :train_data_size, :] #splitting
val_data = data.iloc[train_data_size:train_data_size + val_data_size, :]
test_data = data.iloc[train_data_size + val_data_size:, :]

#getting train data
X_train = train_data[["T"]].values #using .values to convert into numpy array as I am going to use pytorch
y_train = train_data[["x_1","y_1","x_2","y_2"]].values

#getting validation data
X_val = val_data[["T"]].values
y_val = val_data[["x_1","y_1","x_2","y_2"]].values

#getting test data
X_test = test_data[["T"]].values
y_test = test_data[["x_1","y_1","x_2","y_2"]].values

In [35]:
#creating neural model, I will be using FeedForward model for this  with 1 input and 5 outputs
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional  as F

class NN(nn.Module) :
    def __init__(self):
        super().__init__()   #this is how u write constructor in python 
        self.layers = nn.Sequential(
            nn.Linear(1,64),  #1 input layer
            nn.ReLU(),
            nn.Linear(64,64),  #2 hidden layers with 64 64 neurons each activated with ReLU
            nn.ReLU(),
            nn.Linear(64,4)   #4 output layers
        )

    def forward(self,T) :
        return self.layers(T)

model = NN()

In [36]:
#scaling or normalizing the data using StandardScaler
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()  #for X data 
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()  # for y data
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)


In [37]:
#now first I need to convert the data into Pytorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test_scaled)

batch_size = 64 #batch size for batch training

#TensorDataset allows to access to rows from inputs and outputs as tupples/pairs
train_dataset = TensorDataset(X_train_tensor, y_train_tensor) 

#DataLoader splits data into batches for training
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

loss_fn = F.mse_loss # defining loss function

opt = torch.optim.Adam(model.parameters(), lr = 1e-3) #using Adam(Adaptive Moment Estimation) optimizer as it 
                                                       #converges quickly and is widely used in deep learning.


In [38]:
#training of the data
no_epochs = 100 #lets keep it 100
for epoch in range(no_epochs):
    model.train() #for setting the model in training mode
    for x_b, y_b in  train_loader : #x batch and y batch extraction from dataloader
        opt.zero_grad()  #apply zero grad condition
        pred = model(x_b)  #predictions 
        loss = loss_fn(pred, y_b)  #the loss function 
        loss.backward()  #backward step
        opt.step() #used for adjusting weights 

    model.eval() #for setting model in evaluationn mode
    with torch.no_grad():  
        val_pred = model(X_val_tensor) #predictions
        val_loss = loss_fn(val_pred, y_val_tensor) #loss funtions
    #print step to check losses 
    if (epoch+1)%10 == 0 or epoch == 0 :
        print("Epoch [{}/{}], Loss : {:.4f}".format(epoch+1,no_epochs,val_loss))

Epoch [1/100], Loss : 0.0843
Epoch [10/100], Loss : 0.0232
Epoch [20/100], Loss : 0.0199
Epoch [30/100], Loss : 0.0189
Epoch [40/100], Loss : 0.0199
Epoch [50/100], Loss : 0.0171
Epoch [60/100], Loss : 0.0162
Epoch [70/100], Loss : 0.0150
Epoch [80/100], Loss : 0.0152
Epoch [90/100], Loss : 0.0148
Epoch [100/100], Loss : 0.0152


In [39]:
from sklearn.metrics import mean_absolute_error

model.eval() #switch to evaluation mode

with torch.no_grad():
    y_test_pred = model(X_test_tensor) #predictions

#denormaliztion step for calculating mae in original scale
y_test_pred_denorm = scaler_y.inverse_transform(y_test_pred.numpy())
y_test_tensor_denorm = scaler_y.inverse_transform(y_test_tensor.numpy())

mae = mean_absolute_error(y_test_tensor_denorm,y_test_pred_denorm) #mae

#print step
print("Mae for test data is : {}".format(mae))

Mae for test data is : 21103044.0


In [40]:
#find coordinates at t
result = get_coordinates_at_t(t, y0, G, m_1, m_2)
x1, v1, x2, v2 = result[0:2], result[2:4], result[4:6], result[6:8]
#storing x1 and x2 in single numpy array
final_test_case_y = np.concatenate([x1,x2])

#predicting coordinates at time t
final_test_case_x_tensor = torch.tensor([t])

model.eval() #switch to evaluation mode

with torch.no_grad():
    final_test_case_y_pred_tensor = model(final_test_case_x_tensor) #predictions

#calculating mae for this
mae_final = mean_absolute_error(final_test_case_y_pred_tensor.numpy(), final_test_case_y) #calculate to numpy before

#print statement
print("MAE of final test case is: {}".format(mae_final))

MAE of final test case is: 670746965.1675777
